In [1]:
# Importing important librairies
import pyreadstat
import pandas as pd
import os
import numpy as np

In [2]:
# names of the datasets
dfs_names = ["TGIR61FL", "SLIR7AFL","SNIR8BFL","NGIR7BFL","MLIR7AFL", "GMIR81FL","CIIR62FL","NIIR61FL","MRIR71FL","GNIR71FL","GHIR72FL", "BJIR71FL","LBIR7AFL"]

In [5]:
# Loading all the 13 dataset in SPSS format using the pyreadstat package
list_of_df = []
for countries in dfs_names:
    data, _ = pyreadstat.read_sav("Data2/"+countries+".SAV", encoding="latin1")
    list_of_df.append(data)

In [8]:
# Displaying the head of the first table (just for exploration)lem()
list_of_df[10].head()

,CASEID,V000,V001,V002,V003,V004,V005,V006,V007,V008,...,S538E$3,S538E$4,S538E$5,S538E$6,S538G$1,S538G$2,S538G$3,S538G$4,S538G$5,S538G$6
0,1 1 2,GH6,1.0,1.0,2.0,1.0,858896.0,11.0,2014.0,1379.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1 3 2,GH6,1.0,3.0,2.0,1.0,858896.0,11.0,2014.0,1379.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1 5 2,GH6,1.0,5.0,2.0,1.0,858896.0,10.0,2014.0,1378.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1 5 3,GH6,1.0,5.0,3.0,1.0,858896.0,10.0,2014.0,1378.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1 6 2,GH6,1.0,6.0,2.0,1.0,858896.0,10.0,2014.0,1378.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# select the columns that we need in all cross data sets and store the datasets in a new list
filtered_data = []
for data_set in list_of_df:
    filtered_data.append(data_set[["CASEID", "V000","V005", "V007","V010","V013", "V153", "V102", "V119", "V190", "V120", "V121", "V122", "V123", "V124", "V125",
    "V149", "V155", "V106", "V107", "V137", "V138", "V704", "V745A","V745B", "V811", "V743A", "V717","V739"]])

In [13]:
# Add the datasets togheter
concatenated_d = pd.concat(filtered_data, ignore_index=True)

In [220]:
concatenated_d.shape

(179105, 29)

In [221]:
filtered_data2 = []
for i, data_set in enumerate(list_of_df):
    if i in [2, 9, 10]:
        zero_filled_df = pd.DataFrame(0, columns=mm_list, index=data_set.index)
        filtered_data2.append(zero_filled_df)
    else:
        filtered_data2.append(data_set[mm_list])

In [222]:
concatenated_d2 = pd.concat(filtered_data2, ignore_index=True)

In [223]:
concatenated_d2.shape

(179105, 20)

In [ ]:
df = concatenated_d2
columns = concatenated_d2.columns

In [224]:
# We an have answer in the form about materal mortality, these questions are repeated 20 times for every respondant.
# In the below we combine the answers and create 8 columns, each being an answer option for the questions.

values_to_count = [1, 2, 3, 4, 5, 6, 98, 99]
def count_occurrences(row, value):
    return row.tolist().count(value)

for value in values_to_count:
    df[f'occurrences_{value}'] = df.apply(lambda row: count_occurrences(row, value), axis=1)

columns_to_drop = columns
df = df.drop(columns=columns_to_drop)

In [226]:
concatenated_d['index_col'] = range(len(concatenated_d))
df['index_col'] = range(len(df))

# Merge the DataFrames based on the index column
merged_df = pd.merge(concatenated_d, df, on='index_col', how='outer')

In [227]:
# merged_df
merged_df = merged_df.drop(columns=["index_col"])

In [231]:
# New column names
column_names = [
    'CASE_ID', 'COUNTRY', 'SAMPLE_WEIGHT', 'YEAR_INTERVIEW', 'YEAR_DOB', 'AGE_5_YEAR_GROUP',
    'TELEPHONE', 'TYPE_RESIDENCE', 'ELECTRICITY', 'WEALTH_INDEX', 'RADIO', 'TELEVISION',
    'REFRIGERATOR', 'BICYCLE', 'MOTOCYCLE', 'CAR', 'EDUCATIONAL_ATTAINMENT', 'LITERACY',
    'HIGHEST_EDUCATION_LEVEL', 'HIGHEST_YEAR_EDUCATION', 'NUMBER_OF_CHILDREN_UNDER_5',
    'NUMBER_ELIGIBLE_WOMEN', 'HUSBAND_OCCUPATION', 'OWNS_HOUSE', 'OWNS_LAND',
    'CHIL_UNDER_10_WIFE_BEATING', 'OWNS_HEALTH_CARE', 'REPONDANTS_OCCUPATION',"EXPENSE_DECIDER",
    'DEATH_NOT_RELATED', 'DIED_WHILE_PREGNANT', 'DIED_DURING_DELIVERY', 'DIED_SINCE_DELIVERY',
    'DIED_6_WEEKS_AFTER_DELIVERY', 'DIED_2_MONTHS_AFTER_DELIVERY', 'DONT_KNOW', 'MISSING'
]

37

In [233]:
merged_df.columns = column_names

In [236]:
# Export the DataFrame to Excel
file_name = 'full_dataset.xlsx'
merged_df.to_excel(file_name, index=False)

In [ ]:
"""
To prepare the data for dashboarding, we firt load the SPSS country by country using the pyreadstat python package. We then proceeded to deciding the 
columns that we want to keep for our study. We chose columms that provided information about maternal mortality and income inequaty. Consequently we combined all the datasets by 
overlapping them, making them 1 single dataset. In some instances, certain questions are asked 20 times to the same respondant, we used our own python programs to combine the 20 answers into fewer columns.
Finally we downloaded the dataset as an excel database and imported into tableau.
"""
